In [38]:
import concurrent.futures
import time
from urllib.parse import urljoin
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"}

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument('--headless')
chrome_options.add_argument(f'user-agent={headers["User-Agent"]}')
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
url = 'https://www.applied.com/c-brands/c-regal-rexnord/c-stearns-division/105603100bqf/Series-56-000-Disc-Brake/p/101956416'
driver.get(url)
time.sleep(5)
page_sources = driver.page_source
pk = BeautifulSoup(page_sources, 'lxml')


In [39]:
pk.prettify()

'<html class="geolocation localstorage" data-uw-rm-lang="false" data-uw-w-loader="" lang="en">\n <!--<![endif]-->\n <head>\n  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n  <link href="http://www.applied.com//c-brands/c-regal-rexnord/c-stearns-division/105603100bqf/Series-56-000-Disc-Brake/p/101956416" rel="canonical"/>\n  <title>\n   Stearns Division | 105603100BQF | Series 56,000 Disc Brake | Applied\n  </title>\n  <!--OneTrust Cookies Consent Notice Banner-->\n  <script async="" src="https://www.googletagmanager.com/gtag/js?id=AW-528929659&amp;l=dataLayer&amp;cx=c" type="text/javascript">\n  </script>\n  <script async="" src="https://www.googletagmanager.com/gtag/js?id=AW-827755521&amp;l=dataLayer&amp;cx=c" type="text/javascript">\n  </script>\n  <script async="" src="https://www.clarity.ms/s/0.7.47/clarity.js">\n  </script>\n  <script async="" src="https://www.clarity.ms/tag/uet/137019895?insights=1">\n  </script>\n  <script async="" src="https://www.google-analytics.com

# L3_Name

In [34]:
names = [l3.text.strip() for l3 in pk.select("#crumbtrail a")]
l3_Name = 'Home## ' +"## ".join(names)
print('l3_Name.....', l3_Name)

l3_Name..... Home## 


# Title

In [7]:
try:
    product_title = pk.select_one("#pageTitle h1").text.strip()
    product_title
except AttributeError:
    print("'NoneType' object has no attribute 'text'")

'NoneType' object has no attribute 'text'


## Mpn

In [46]:
try:
    mpn = 'rp_' + pk.select_one("div.dc-details-numbers").text.strip().split('#')[1].strip()
    print(mpn)
except AttributeError:
    print("'NoneType' object has no attribute 'get'")



rp_101956416


## short desc

In [6]:
try:
    short = pk.select_one("div.product .slo-product-info p.my-md").text.strip()
except AttributeError:
    print("NoneType' object has no attribute 'text")

NoneType' object has no attribute 'text


## Description_1

In [7]:
description = [des.text.strip() for des in pk.select("div#content p")[2:-1]]
print(description)

['Knife Gate Valves from Tecofi, a French valve manufacturer specialising in valves for Water Treatment, Biogas and General Industrial Process.', '', 'The Series 43022 is a Bi Directional cast iron knife gate valve, with NBR seal with a stainless steel gate, suitable for fitting between PN10 flanges and operated by a hand wheel operator. Available from DN50 to DN400 and deliveries for non stock items usually 7-10 days from date of order.', 'Suitable for use on WWT applications, Biogas, Slurry and other Industrial Process applications.', "Don't forget your accessories, we can supply a wide range of accessories including limit switches, flanges and gaskets.", 'To buy a priced product online, please click on it in the table below.Full pricing, discounts, datasheets and extended information will also be available on that page.If you would like bulk or project pricing, technical support, or have any other inquiries, please use the RFQ or call buttons to discuss your requirements with a memb

## Description_2

In [8]:
description_2 = [feature.text.strip() for feature in pk.select("section.product-section.product-features-list ul li")]
if not description_2:
    description_2 = ['No Feature available ']
description_2

['No Feature available ']

## Price 

In [22]:
# pr_qty = ''
# try:
#     pr_qty = pk.select_one("div.prodData .umDiv").text.strip().split(':')[1].strip()
# except (AttributeError, TypeError, IndexError, Exception) as e:
#     print(f"An error occurred: {e} pr_qty -----: {pr_qty}")
    
# -------------------------------------------------- Price -----------------------------------------------------
product_price = ''
try:
    product_price = pk.select_one("div.dc-price-box").get_text(strip=True, separator='/ ')
    price = {'list_price': product_price, 'qty': '1', 'moq': '1'}
except (AttributeError, TypeError, IndexError, Exception) as e:
    print(f"An error occurred: {e} Retail_Price -----: {product_price}")
print(price)

{'list_price': '$246.48/ each/ One-time purchase', 'qty': '1', 'moq': '1'}


## description_2

In [10]:
try:
    description_2 = [feature.text.strip() for feature in pk.select("#spec ul li")]
    if not description_2:
        raise ValueError("No description_2 found")
    print('description_2.....', description_2)
except (AttributeError, TypeError, IndexError, Exception) as e:
    description_2 = []
    print(f"An error occurred: {e} description_2 -----: {description_2}")

An error occurred: No description_2 found description_2 -----: []


## Images

In [11]:
images = [{"src": img.get('src'), "alt": img.get('alt')} for img in pk.select("div.mainProductImage img")]
print(images)

[{'src': '/UserControls/ThumbNailGenerator.aspx?img=/uploads/images/categories/vg3400-03-jpg&w=250&h=250&ar=true', 'alt': 'Knife Gate Valve Bi Directional Pneumatic Actuated Tecofi'}]


## Specs

In [12]:
attri_name = []
attri_value = []
row_count = 0
for table in pk.select("section#specifications span"):
    tab = table.text.strip()
    row_count += 1
    if row_count % 2 != 0:
        attri_name.append(tab)
    else:
        attri_value.append(tab)
specs = [{name:value for name, value in zip(attri_name, attri_value)}]
print(specs)
    


[{}]


In [30]:
tables = pk.select("section#specifications span")
it = iter(tables)
specs = [{name.text.strip(): value.text.strip()} for name, value in zip(it, it)]
if not specs:
    specs = ['No Specs available ']
print(specs)


['No Specs Available ']


In [13]:
try:
    Packed_per = pk.select_one("#add-to-cart-row").text.strip().replace('\n', '').split('Full')[0].replace('-+Packed per', '').strip()
    print(Packed_per)
except AttributeError:
    print("'NoneType' object has no attribute 'text'")

'NoneType' object has no attribute 'text'


In [14]:
try:
    in_stocks = pk.select_one("div.adp-stock span").text.strip()
    print(in_stocks)
except AttributeError:
    print("'NoneType' object has no attribute 'text'")

'NoneType' object has no attribute 'text'


## Logo

In [15]:
logo = pk.select_one("div#content p img").get('src')
print(logo)

https://www.allvalves.co.uk/uploads/images/2019%20Website%20Images/Logos/Tecofi_logo.jpg


## Datasheet

In [16]:
datasheet = [pdf.get('href') for pdf in pk.select("#downloads a") if pdf.get('href') and 'pdf' in pdf.get('href').lower()]
if not datasheet:
    datasheet = ['No datasheet available']
print(datasheet)


['No datasheet available']


## Related

In [35]:
source_of_related = [source.get('href') for source in pk.select('div.related-products-grid header a')]
if not source_of_related:
    source_of_related = ['No related url available']
print(source_of_related)

['No related url available']


## Miscellaneous

In [37]:
Ready_To_Ship = ''
try:
    Ready_To_Ship = pk.select_one("div.product__stock-message").text.strip()
except (AttributeError, TypeError, IndexError, Exception) as e:
    print(f"An error occurred: {e} Ready_To_Ship -----: {Ready_To_Ship}")
print(Ready_To_Ship)

An error occurred: 'NoneType' object has no attribute 'text' Ready_To_Ship -----: 



In [19]:
text = ""
for element in pk.find_all(['font', 'img']):
    print(element)
#     if element.name == 'font':
#         text += element.get_text()  # Extract text from <font> tag
#     elif element.name == 'img':
#         text += element.get('alt')  # Extract alt attribute from <img> tag
#     if element.next_sibling and isinstance(element.next_sibling, str):
#         text += element.next_sibling.strip()  # Get any text next to the element

# # Print the combined text
# print(text)




<img border="0" src="/images/shell/allvalves_original.png"/>
<img alt="Knife Gate Valve Bi Directional Pneumatic Actuated Tecofi" border="0" src="/UserControls/ThumbNailGenerator.aspx?img=/uploads/images/categories/vg3400-03-jpg&amp;w=250&amp;h=250&amp;ar=true"/>
<img alt="" src="https://www.allvalves.co.uk/uploads/images/2019%20Website%20Images/Logos/Tecofi_logo.jpg" style="left: 729px; top: 205px; width: 304px; height: 110px;"/>
<font size="3"><strong>Stock: </strong></font>
<img alt="Stock available next day" class="status-led" src="/images/green-dot.png"/>
<img alt="Stock available next day" class="status-led" src="/images/orange-dot.png"/>
<img alt="Stock available next day" class="status-led" src="/images/red-dot.png"/>
<img alt="All Valves Online - Always Online!" src="/uploads/images/2019%20Website%20Images/Logos/allvalves-logo-round-small.png" style="width: 50px; height: 50px;"/>
<font size="7"><span style="color: rgb(43, 121, 190);">The Valve Actuation Specialists</span></fon